<a href="https://colab.research.google.com/github/12032004/main-project-phase1/blob/main/social_media_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
from google.colab import files

In [15]:
uploaded=files.upload()

Saving data_sources.xlsx to data_sources.xlsx


In [17]:
df = pd.DataFrame()

In [18]:
df=pd.read_excel('data_sources.xlsx')

In [19]:
print(df.head())

                                               Paper  \
0  CLPsych 2015 Shared Task: Depression and PTSD ...   
1       Quantifying Mental Health Signals in Twitter   
2  From ADHD to SAD: Analyzing the Language of Me...   
3  Towards Augmenting Crisis Counselor Training b...   
4  Characterizing and Predicting Postpartum Depre...   

                                             Authors  Year  \
0  Coppersmith, Dredze, Harman, Hollingshead, Mit...  2015   
1                        Coppersmith, Dredze, Harman  2014   
2          Coppersmith, Dredze, Harman, Hollingshead  2015   
3                              DeMasi, Hearst, Recht  2019   
4                De Choudhury, Counts, Horvitz, Hoff  2014   

                              Platform  \
0                              Twitter   
1                              Twitter   
2                              Twitter   
3  Synthetic Crisis Text Conversations   
4                             Facebook   

                                     

In [20]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

In [26]:
print(df.columns)

Index(['Paper', 'Authors', 'Year', 'Platform', 'Target Outcomes',
       'Labeling Methodology', 'Size', 'Availability', 'Additional Comments',
       'Dataset Link (if any)', 'Reference Link'],
      dtype='object')


In [27]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Paper'].tolist(), df['Target Outcomes'].tolist(), test_size=0.2, random_state=42)

In [28]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [33]:
import math

train_labels = [int(label) if isinstance(label, float) and not math.isnan(label) else label for label in train_labels]
val_labels = [int(label) if isinstance(label, float) and not math.isnan(label) else label for label in val_labels]

In [35]:
train_labels = [str(label) for label in train_labels]
val_labels = [str(label) for label in val_labels]

train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "labels": train_labels})
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "labels": val_labels})

In [37]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(set(df["Target Outcomes"]))) # Changed "label" to "Target Outcomes"

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [39]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset)

In [42]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

print("Training complete! Model saved.")

Training complete! Model saved.
